# 來寫新策略 -- 透過backtrader
1. 依據不同sector來選股
2. 試著找出強勢sector看簡單策略有沒有效果

In [1]:
import pymongo
import numpy as np
import pandas as pd
import datetime
import os.path
import sys

In [2]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import backtrader as bt

class Strategy(bt.Strategy):

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % 
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                    
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

            if order.issell():
                self.log(
                    'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % 
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

            self.bar_executed = len(self)
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def nodify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                (trade.pnl, trade.pnlcomm))


    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])

        if self.order:
            return

        if not self.position:

            if self.dataclose[0] < self.dataclose[-1]:

                if self.dataclose[-1] < self.dataclose[-2]:

                    self.log('BUY CREATE, %.2f' % self.dataclose[0])
                    self.order = self.buy()

        else:
            if len(self) >= (self.bar_executed + 5):

                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                self.order = self.sell()


if __name__ == '__main__':
    cerebro = bt.Cerebro(stdstats=True)
    cerebro.addstrategy(Strategy)
    
    client = pymongo.MongoClient()
    db = client['Fields']

    def ohlcv(from_date: datetime.datetime=datetime.datetime(2022, 1, 1),
                to_date: datetime.datetime=datetime.datetime.today()) -> dict:
        data = {}
        o = pd.DataFrame(db['還原開盤價'].find({'日期': {'$gte': from_date, '$lte': to_date}}, {'_id': 0})).set_index('日期')
        h = pd.DataFrame(db['還原開盤價'].find({'日期': {'$gte': from_date, '$lte': to_date}}, {'_id': 0})).set_index('日期')
        l = pd.DataFrame(db['還原開盤價'].find({'日期': {'$gte': from_date, '$lte': to_date}}, {'_id': 0})).set_index('日期')
        c = pd.DataFrame(db['還原開盤價'].find({'日期': {'$gte': from_date, '$lte': to_date}}, {'_id': 0})).set_index('日期')
        v = pd.DataFrame(db['還原開盤價'].find({'日期': {'$gte': from_date, '$lte': to_date}}, {'_id': 0})).set_index('日期')
        for stk in o.columns:
            data[stk] = pd.concat([o.loc[:, stk], h.loc[:, stk], l.loc[:, stk], c.loc[:, stk], v.loc[:, stk]], axis=1)
            data[stk].columns = ['open', 'high', 'low', 'close', 'volume']
        return data

    dataframe = ohlcv()
    # Create a Data Feed
    data = bt.feeds.PandasData(dataname=dataframe['2330'])
    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000000.0)
    cerebro.broker.setcommission(commission=0.002)
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    cerebro.plot(iplot=False)
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000000.00
2022-01-03, Close, 612.62
2022-01-04, Close, 638.35
2022-01-05, Close, 662.11
2022-01-06, Close, 631.43
2022-01-07, Close, 636.37
2022-01-10, Close, 621.53
2022-01-11, Close, 639.34
2022-01-12, Close, 650.23
2022-01-13, Close, 651.22
2022-01-14, Close, 666.07
2022-01-17, Close, 677.94
2022-01-18, Close, 671.01
2022-01-19, Close, 645.28
2022-01-19, BUY CREATE, 645.28
2022-01-20, BUY EXECUTED, Price: 643.30, Cost: 643.30, Comm 1.29
2022-01-20, Close, 643.30
2022-01-21, Close, 637.36
2022-01-24, Close, 632.42
2022-01-25, Close, 636.37
2022-01-26, Close, 628.46
2022-02-07, Close, 637.36
2022-02-07, SELL CREATE, 637.36
2022-02-08, SELL EXECUTED, Price: 638.35, Cost: 643.30, Comm 1.28
2022-02-08, Close, 638.35
2022-02-09, Close, 628.46
2022-02-10, Close, 632.42
2022-02-11, Close, 639.34
2022-02-14, Close, 633.41
2022-02-15, Close, 626.48
2022-02-15, BUY CREATE, 626.48
2022-02-16, BUY EXECUTED, Price: 638.35, Cost: 638.35, Comm 1.28
2022-02-16, Close, 63